# Data Extraction Module for Pricing & Offers System


## Imports


In [2]:
import os
import numpy as np
import pandas as pd
import snowflake.connector
import setup_environment_2


## Variables


In [ ]:
# Region
REGION = "Egypt"

# Snowflake Warehouse
WAREHOUSE = "COMPUTE_WH"

# Date Variables
from datetime import datetime, timedelta
TODAY = datetime.now().date()
YESTERDAY = TODAY - timedelta(days=1)


## Initialize Environment


In [3]:
setup_environment_2.initialize_env()


NoCredentialsError: Unable to locate credentials

## Warehouse & Cohort Mapping


In [ ]:
# Warehouse Mapping: (region, warehouse_name, warehouse_id, cohort_id)
WAREHOUSE_MAPPING = [
    ('Cairo', 'El-Marg', 38, 700),
    ('Cairo', 'Mostorod', 1, 700),
    ('Giza', 'Barageel', 236, 701),
    ('Giza', 'Sakkarah', 962, 701),
    ('Delta West', 'El-Mahala', 337, 703),
    ('Delta West', 'Tanta', 8, 703),
    ('Delta East', 'Mansoura FC', 339, 704),
    ('Delta East', 'Sharqya', 170, 704),
    ('Upper Egypt', 'Assiut FC', 501, 1124),
    ('Upper Egypt', 'Bani sweif', 401, 1126),
    ('Upper Egypt', 'Menya Samalot', 703, 1123),
    ('Upper Egypt', 'Sohag', 632, 1125),
    ('Alexandria', 'Khorshed Alex', 797, 702),
]

# Region to Cohort Mapping
REGION_COHORT_MAPPING = {
    'Cairo': 700,
    'Giza': 701,
    'Delta West': 703,
    'Delta East': 704,
    'Upper Egypt': 1124,
    'Alexandria': 702,
}

# All Cohort IDs
COHORT_IDS = [700, 701, 702, 703, 704, 1123, 1124, 1125, 1126]


## Snowflake Query Function


In [ ]:
def query_snowflake(query):
    """Execute a query on Snowflake and return results as DataFrame."""
    con = snowflake.connector.connect(
        user=os.environ["SNOWFLAKE_USERNAME"],
        account=os.environ["SNOWFLAKE_ACCOUNT"],
        password=os.environ["SNOWFLAKE_PASSWORD"],
        database=os.environ["SNOWFLAKE_DATABASE"]
    )
    try:
        cur = con.cursor()
        cur.execute("USE WAREHOUSE COMPUTE_WH")
        cur.execute(query)
        data = cur.fetchall()
        columns = [desc[0].lower() for desc in cur.description]  # Get column names from cursor
        return pd.DataFrame(data, columns=columns)
    except Exception as e:
        print(f"Snowflake Error: {e}")
        return pd.DataFrame()
    finally:
        cur.close()
        con.close()


In [ ]:
def get_snowflake_timezone():
    """Get the current timezone from Snowflake."""
    query = "SHOW PARAMETERS LIKE 'TIMEZONE'"
    result = query_snowflake(query)
    return result[1].values[0] if len(result) > 0 else "UTC"


## Helper Functions


In [ ]:
def get_warehouse_df():
    """Get warehouse mapping as DataFrame."""
    return pd.DataFrame(
        WAREHOUSE_MAPPING,
        columns=['region', 'warehouse', 'warehouse_id', 'cohort_id']
    )


def get_cohort_by_region(region):
    """Get cohort ID for a given region."""
    return REGION_COHORT_MAPPING.get(region)


def convert_to_numeric(df):
    """Convert DataFrame columns to numeric where possible."""
    df.columns = df.columns.str.lower()
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='ignore')
    return df


## Get Snowflake Timezone


In [ ]:
TIMEZONE = get_snowflake_timezone()
print(f"Snowflake timezone: {TIMEZONE}")


## Market Prices Extraction Queries
Queries for external market price data:
1. **Ben Soliman Prices** - Competitor reference prices
2. **Marketplace Prices** - Min, Max, Mod prices from marketplace
3. **Scrapped Data** - Competitor prices from scraping


In [ ]:
# =============================================================================
# 1. BEN SOLIMAN PRICES QUERY
# =============================================================================
BEN_SOLIMAN_QUERY = f'''
WITH lower as (
    select distinct product_id, sku, new_d*bs_price as ben_soliman_price, INJECTION_DATE
    from (
        select maxab_product_id as product_id, maxab_sku as sku, INJECTION_DATE, wac1, wac_p,
            (bs_price/bs_unit_count) as bs_price, diff, cu_price,
            case when p1 > 1 then child_quantity else 0 end as scheck,
            round(p1/2)*2 as p1, p2,
            case when (ROUND(p1 / scheck) * scheck) = 0 then p1 else (ROUND(p1 / scheck) * scheck) end as new_d
        from (
            select sm.*, wac1, wac_p, 
                abs((bs_price/bs_unit_count)-(wac_p*maxab_basic_unit_count))/(wac_p*maxab_basic_unit_count) as diff,
                cpc.price as cu_price, pup.child_quantity,
                round((cu_price/(bs_price/bs_unit_count))) as p1, 
                round(((bs_price/bs_unit_count)/cu_price)) as p2
            from materialized_views.savvy_mapping sm 
            join finance.all_cogs f on f.product_id = sm.maxab_product_id 
                and CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP()) between f.from_Date and f.to_date
            join PACKING_UNIT_PRODUCTS pu on pu.product_id = sm.maxab_product_id and pu.IS_BASIC_UNIT = 1 
            join cohort_product_packing_units cpc on cpc.PRODUCT_PACKING_UNIT_ID = pu.id and cohort_id = 700 
            join packing_unit_products pup on pup.product_id = sm.maxab_product_id and pup.is_basic_unit = 1  
            where bs_price is not null 
                and INJECTION_DATE::date >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date - 5 
                and diff > 0.3 and p1 > 1
        )
    )
    qualify max(INJECTION_DATE) over(partition by product_id) = INJECTION_DATE
),

m_bs as (
    select z.* from (
        select maxab_product_id as product_id, maxab_sku as sku, avg(bs_final_price) as ben_soliman_price, INJECTION_DATE
        from (
            select *, row_number() over(partition by maxab_product_id order by diff) as rnk_2 
            from (
                select *, (bs_final_price-wac_p)/wac_p as diff_2 
                from (
                    select *, bs_price/maxab_basic_unit_count as bs_final_price 
                    from (
                        select *, row_number() over(partition by maxab_product_id, maxab_pu order by diff) as rnk 
                        from (
                            select *, max(INJECTION_DATE::date) over(partition by maxab_product_id, maxab_pu) as max_date
                            from (
                                select sm.*, wac1, wac_p, 
                                    abs(bs_price-(wac_p*maxab_basic_unit_count))/(wac_p*maxab_basic_unit_count) as diff 
                                from materialized_views.savvy_mapping sm 
                                join finance.all_cogs f on f.product_id = sm.maxab_product_id 
                                    and CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP()) between f.from_Date and f.to_date
                                where bs_price is not null 
                                    and INJECTION_DATE::date >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date - 5 
                                    and diff < 0.3
                            )
                            qualify max_date = INJECTION_DATE
                        ) qualify rnk = 1 
                    )
                ) where diff_2 between -0.5 and 0.5 
            ) qualify rnk_2 = 1 
        ) group by all
    ) z 
    join finance.all_cogs f on f.product_id = z.product_id 
        and CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP()) between f.from_Date and f.to_date
    where ben_soliman_price between f.wac_p*0.8 and f.wac_p*1.3
)

select product_id, avg(ben_soliman_price) as ben_soliman_price
from (
    select * from (
        select * from m_bs 
        union all
        select * from lower
    )
    qualify max(INJECTION_DATE) over(partition by product_id) = INJECTION_DATE
)
group by all
'''


In [ ]:
# =============================================================================
# 2. MARKETPLACE PRICES QUERY (with region fallback)
# =============================================================================
MARKETPLACE_PRICES_QUERY = f'''
WITH MP as (
    select region, product_id,
        min(min_price) as min_price, min(max_price) as max_price,
        min(mod_price) as mod_price, min(true_min) as true_min, min(true_max) as true_max
    from (
        select mp.region, mp.product_id, mp.pu_id,
            min_price/BASIC_UNIT_COUNT as min_price,
            max_price/BASIC_UNIT_COUNT as max_price,
            mod_price/BASIC_UNIT_COUNT as mod_price,
            TRUE_MIN_PRICE/BASIC_UNIT_COUNT as true_min,
            TRUE_MAX_PRICE/BASIC_UNIT_COUNT as true_max
        from materialized_views.marketplace_prices mp 
        join packing_unit_products pup on pup.product_id = mp.product_id and pup.packing_unit_id = mp.pu_id
        join finance.all_cogs f on f.product_id = mp.product_id 
            and CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP()) between f.from_date and f.to_date
        where least(min_price, mod_price) between wac_p*0.9 and wac_p*1.3 
    )
    group by all 
),

region_mapping AS (
    SELECT * FROM (VALUES
        ('Delta East', 'Delta West'), ('Delta West', 'Delta East'),
        ('Alexandria', 'Cairo'), ('Alexandria', 'Giza'),
        ('Upper Egypt', 'Cairo'), ('Upper Egypt', 'Giza'),
        ('Cairo', 'Giza'), ('Giza', 'Cairo'),
        ('Delta West', 'Cairo'), ('Delta East', 'Cairo'),
        ('Delta West', 'Giza'), ('Delta East', 'Giza')
    ) AS region_mapping(region, fallback_region)
),

all_regions as (
    SELECT * FROM (VALUES
        ('Cairo'), ('Giza'), ('Delta West'), ('Delta East'), ('Upper Egypt'), ('Alexandria')
    ) AS x(region)
),

full_data as (
    select products.id as product_id, ar.region
    from products, all_regions ar
    where activation = 'true'
)

select region, product_id,
    min(final_min_price) as final_min_price, 
    min(final_max_price) as final_max_price,
    min(final_mod_price) as final_mod_price, 
    min(final_true_min) as final_true_min,
    min(final_true_max) as final_true_max
from (
    SELECT distinct w.region, w.product_id,
        COALESCE(m1.min_price, m2.min_price) AS final_min_price,
        COALESCE(m1.max_price, m2.max_price) AS final_max_price,
        COALESCE(m1.mod_price, m2.mod_price) AS final_mod_price,
        COALESCE(m1.true_min, m2.true_min) AS final_true_min,
        COALESCE(m1.true_max, m2.true_max) AS final_true_max
    FROM full_data w
    LEFT JOIN MP m1 ON w.region = m1.region and w.product_id = m1.product_id
    LEFT JOIN region_mapping rm ON w.region = rm.region
    LEFT JOIN MP m2 ON rm.fallback_region = m2.region AND w.product_id = m2.product_id
)
where final_min_price is not null 
group by all
'''


In [ ]:
# =============================================================================
# 3. SCRAPPED DATA QUERY (Competitor prices from scraping)
# =============================================================================
SCRAPPED_DATA_QUERY = f'''
select product_id, region,
    MIN(market_price) AS min_scrapped,
    PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY market_price) AS scrapped25,
    PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY market_price) AS scrapped50,
    PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY market_price) AS scrapped75,
    MAX(market_price) AS max_scrapped
from (
    select distinct cmp.*, max(date) over(partition by region, cmp.product_id, competitor) as max_date
    from MATERIALIZED_VIEWS.CLEANED_MARKET_PRICES cmp
    join finance.all_cogs f on f.product_id = cmp.product_id 
        and CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP()) between f.from_date and f.to_date 
    where date >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date - 7 
        and MARKET_PRICE between f.wac_p * 0.8 and wac_p * 1.3
    qualify date = max_date 
)
group by all
'''


In [ ]:
## Additional Data Queries (Sales, Groups, WAC)


In [ ]:
# =============================================================================
# 4. PRODUCT BASE DATA QUERY (product_id, sku, brand, cat, wac1, wac_p, current_price)
# =============================================================================
PRODUCT_BASE_QUERY = f'''
WITH skus_prices AS (
    WITH local_prices AS (
        SELECT  
            CASE 
                WHEN cpu.cohort_id IN (700, 695) THEN 'Cairo'
                WHEN cpu.cohort_id IN (701) THEN 'Giza'
                WHEN cpu.cohort_id IN (704, 698) THEN 'Delta East'
                WHEN cpu.cohort_id IN (703, 697) THEN 'Delta West'
                WHEN cpu.cohort_id IN (696, 1123, 1124, 1125, 1126) THEN 'Upper Egypt'
                WHEN cpu.cohort_id IN (702, 699) THEN 'Alexandria'
            END AS region,
            cohort_id,
            pu.product_id,
            pu.packing_unit_id,
            pu.basic_unit_count,
            AVG(cpu.price) AS price
        FROM cohort_product_packing_units cpu
        JOIN PACKING_UNIT_PRODUCTS pu ON pu.id = cpu.product_packing_unit_id
        WHERE cpu.cohort_id IN (700,701,702,703,704,695,696,697,698,699,1123,1124,1125,1126)
            AND cpu.created_at::date <> '2023-07-31'
            AND cpu.is_customized = TRUE
        GROUP BY ALL
    ),
    
    live_prices AS (
        SELECT 
            region, cohort_id, product_id, 
            pu_id AS packing_unit_id, 
            buc AS basic_unit_count, 
            NEW_PRICE AS price
        FROM materialized_views.DBDP_PRICES
        WHERE created_at = CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date
            AND DATE_PART('hour', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::time) 
                BETWEEN SPLIT_PART(time_slot, '-', 1)::int AND (SPLIT_PART(time_slot, '-', 1)::int) + 1
            AND cohort_id IN (700,701,702,703,704,695,696,697,698,699,1123,1124,1125,1126)
    ),
    
    prices AS (
        SELECT *
        FROM (
            SELECT *, 1 AS priority FROM live_prices
            UNION ALL
            SELECT *, 2 AS priority FROM local_prices
        )
        QUALIFY ROW_NUMBER() OVER (PARTITION BY region, cohort_id, product_id, packing_unit_id ORDER BY priority) = 1
    )
    
    SELECT region, cohort_id, product_id, price
    FROM prices
    WHERE basic_unit_count = 1
        AND ((product_id = 1309 AND packing_unit_id = 2) OR (product_id <> 1309))
)

SELECT distinct
    region, cohort_id, p.product_id,
    CONCAT(products.name_ar, ' ', products.size, ' ', product_units.name_ar) AS sku,
    b.name_ar AS brand,
    cat.name_ar AS cat,
    wac1, wac_p, p.price as current_price
FROM skus_prices p
JOIN finance.all_cogs c ON c.product_id = p.product_id 
    AND CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP()) BETWEEN c.from_date AND c.to_date
JOIN products ON products.id = p.product_id
JOIN categories cat ON cat.id = products.category_id
JOIN brands b ON b.id = products.brand_id
JOIN product_units ON product_units.id = products.unit_id
WHERE wac1 > 0 AND wac_p > 0
GROUP BY ALL
'''

# =============================================================================
# 5. SALES DATA QUERY (120-day NMV by cohort/product)
# =============================================================================
SALES_QUERY = f'''
SELECT DISTINCT cpc.cohort_id, pso.product_id,
    CONCAT(products.name_ar,' ',products.size,' ',product_units.name_ar) as sku,
    brands.name_ar as brand, categories.name_ar as cat,
    sum(pso.total_price) as nmv
FROM product_sales_order pso
JOIN sales_orders so ON so.id = pso.sales_order_id
JOIN COHORT_PRICING_CHANGES cpc ON cpc.id = pso.COHORT_PRICING_CHANGE_id
JOIN products ON products.id = pso.product_id
JOIN brands ON products.brand_id = brands.id 
JOIN categories ON products.category_id = categories.id
JOIN product_units ON product_units.id = products.unit_id 
WHERE so.created_at::date BETWEEN CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date - 120 
    AND CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date - 1 
    AND so.sales_order_status_id NOT IN (7, 12)
    AND so.channel IN ('telesales', 'retailer')
    AND pso.purchased_item_count <> 0
    AND cpc.cohort_id IN (700,701,702,703,704,1123,1124,1125,1126)
GROUP BY ALL
'''

# =============================================================================
# 6. MARGIN STATS QUERY (STD and average margins)  
# =============================================================================
MARGIN_STATS_QUERY = f'''
select product_id, cohort_id, 
    (0.6*product_std) + (0.3*brand_std) + (0.1*cat_std) as std, 
    avg_margin
from (
    select product_id, cohort_id, 
        stddev(product_margin) as product_std, 
        stddev(brand_margin) as brand_std,
        stddev(cat_margin) as cat_std, 
        avg(product_margin) as avg_margin
    from (
        select distinct product_id, order_date, cohort_id,
            (nmv-cogs_p)/nmv as product_margin, 
            (brand_nmv-brand_cogs)/brand_nmv as brand_margin,
            (cat_nmv-cat_cogs)/cat_nmv as cat_margin
        from (
            SELECT DISTINCT so.created_at::date as order_date, cpc.cohort_id, pso.product_id,
                brands.name_ar as brand, categories.name_ar as cat,
                sum(COALESCE(f.wac_p,0) * pso.purchased_item_count * pso.basic_unit_count) as cogs_p,
                sum(pso.total_price) as nmv,
                sum(nmv) over(partition by order_date, cat, brand) as brand_nmv,
                sum(cogs_p) over(partition by order_date, cat, brand) as brand_cogs,
                sum(nmv) over(partition by order_date, cat) as cat_nmv,
                sum(cogs_p) over(partition by order_date, cat) as cat_cogs
            FROM product_sales_order pso
            JOIN sales_orders so ON so.id = pso.sales_order_id   
            JOIN COHORT_PRICING_CHANGES cpc on cpc.id = pso.cohort_pricing_change_id
            JOIN products on products.id = pso.product_id
            JOIN brands on products.brand_id = brands.id 
            JOIN categories ON products.category_id = categories.id
            JOIN finance.all_cogs f ON f.product_id = pso.product_id
                AND f.from_date::date <= so.created_at::date AND f.to_date::date > so.created_at::date
            WHERE so.created_at::date between 
                date_trunc('month', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date - 120) 
                and CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date
                AND so.sales_order_status_id not in (7,12)
                AND so.channel IN ('telesales','retailer')
                AND pso.purchased_item_count <> 0
            GROUP BY ALL
        )
    ) group by all 
)
'''

# =============================================================================
# 7. TARGET MARGINS QUERY
# =============================================================================
TARGET_MARGINS_QUERY = f'''
WITH cat_brand_target as (
    SELECT DISTINCT cat, brand, margin as target_bm
    FROM performance.commercial_targets cplan
    QUALIFY CASE 
        WHEN DATE_TRUNC('month', MAX(DATE) OVER()) = DATE_TRUNC('month', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date) 
        THEN DATE_TRUNC('month', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date)
        ELSE DATE_TRUNC('month', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date - INTERVAL '1 month') 
    END = DATE_TRUNC('month', date)
),
cat_target as (
    select cat, sum(target_bm * (target_nmv/cat_total)) as cat_target_margin
    from (
        select *, sum(target_nmv) over(partition by cat) as cat_total
        from (
            select cat, brand, avg(target_bm) as target_bm, sum(target_nmv) as target_nmv
            from (
                SELECT DISTINCT date, city as region, cat, brand, margin as target_bm, nmv as target_nmv
                FROM performance.commercial_targets cplan
                QUALIFY CASE 
                    WHEN DATE_TRUNC('month', MAX(DATE) OVER()) = DATE_TRUNC('month', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date) 
                    THEN DATE_TRUNC('month', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date)
                    ELSE DATE_TRUNC('month', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date - INTERVAL '1 month') 
                END = DATE_TRUNC('month', date)
            ) group by all
        )
    ) group by all 
)
SELECT DISTINCT cbt.cat, cbt.brand, cbt.target_bm, ct.cat_target_margin
FROM cat_brand_target cbt
LEFT JOIN cat_target ct ON ct.cat = cbt.cat
'''


In [ ]:
## Execute All Queries


In [ ]:
# =============================================================================
# Execute all queries
# =============================================================================
print("Loading data from Snowflake...")

# 1. Ben Soliman Prices
print("  1. Loading Ben Soliman prices...")
df_ben_soliman = query_snowflake(BEN_SOLIMAN_QUERY)
df_ben_soliman = convert_to_numeric(df_ben_soliman)
print(f"     Loaded {len(df_ben_soliman)} Ben Soliman price records")

# 2. Marketplace Prices
print("  2. Loading marketplace prices...")
df_marketplace = query_snowflake(MARKETPLACE_PRICES_QUERY)
df_marketplace = convert_to_numeric(df_marketplace)
print(f"     Loaded {len(df_marketplace)} marketplace price records")

# 3. Scrapped Data
print("  3. Loading scrapped data...")
df_scrapped = query_snowflake(SCRAPPED_DATA_QUERY)
df_scrapped = convert_to_numeric(df_scrapped)
print(f"     Loaded {len(df_scrapped)} scrapped price records")

# 4. Product Base Data (product_id, sku, brand, cat, wac1, wac_p, current_price)
print("  4. Loading product base data...")
df_product_base = query_snowflake(PRODUCT_BASE_QUERY)
df_product_base = convert_to_numeric(df_product_base)
print(f"     Loaded {len(df_product_base)} product base records")

# 5. Sales Data
print("  5. Loading sales data...")
df_sales = query_snowflake(SALES_QUERY)
df_sales = convert_to_numeric(df_sales)
print(f"     Loaded {len(df_sales)} sales records")

# 6. Margin Stats
print("  6. Loading margin stats...")
df_margin_stats = query_snowflake(MARGIN_STATS_QUERY)
df_margin_stats = convert_to_numeric(df_margin_stats)
print(f"     Loaded {len(df_margin_stats)} margin stat records")

# 7. Target Margins
print("  7. Loading target margins...")
df_targets = query_snowflake(TARGET_MARGINS_QUERY)
df_targets = convert_to_numeric(df_targets)
print(f"     Loaded {len(df_targets)} target margin records")

# 8. Product Groups (from PostgreSQL)
print("  8. Loading product groups...")
df_groups = setup_environment_2.dwh_pg_query(
    "SELECT * FROM materialized_views.sku_commercial_groups", 
    columns=['product_id', 'group']
)
df_groups.columns = df_groups.columns.str.lower()
df_groups = convert_to_numeric(df_groups)
print(f"     Loaded {len(df_groups)} group records")

print("\nAll queries completed!")
print(f"\n{'='*60}")
print("df_product_base DataFrame available with columns:")
print("  - region, cohort_id, product_id, sku, brand, cat, wac1, wac_p, current_price")
print(f"{'='*60}")


In [ ]:
# =============================================================================
# Build base pricing_data DataFrame - Start with PRODUCT_BASE (all our SKUs)
# =============================================================================
print("Building pricing_data DataFrame...")

# =============================================================================
# Step 1: Start with df_product_base as the MAIN dataframe (all our SKUs)
# =============================================================================
print("  Step 1: Starting with product base (all SKUs)...")
pricing_data = df_product_base.copy()
print(f"     Product base: {len(pricing_data)} records")

# =============================================================================
# Step 2: Add warehouse mapping (warehouse_id and warehouse name)
# =============================================================================
print("  Step 2: Adding warehouse mapping...")
warehouse_df = get_warehouse_df()
pricing_data = pricing_data.merge(
    warehouse_df[['cohort_id', 'warehouse_id', 'warehouse']], 
    on='cohort_id', 
    how='left'
)

# =============================================================================
# Step 3: LEFT JOIN market price data (marketplace, scrapped, ben_soliman)
# =============================================================================
print("  Step 3: Left joining market price data...")

# LEFT JOIN marketplace prices (by region + product_id)
pricing_data = pricing_data.merge(
    df_marketplace, 
    on=['region', 'product_id'], 
    how='left'
)
print(f"     After marketplace join: {len(pricing_data)} records")

# LEFT JOIN scrapped data (by region + product_id)
pricing_data = pricing_data.merge(
    df_scrapped, 
    on=['region', 'product_id'], 
    how='left'
)
print(f"     After scrapped join: {len(pricing_data)} records")

# LEFT JOIN Ben Soliman prices (by product_id only - same price for all regions)
pricing_data = pricing_data.merge(
    df_ben_soliman, 
    on='product_id', 
    how='left'
)
print(f"     After Ben Soliman join: {len(pricing_data)} records")

# =============================================================================
# Step 4: LEFT JOIN supporting data (sales, margins, targets, groups)
# =============================================================================
print("  Step 4: Left joining supporting data...")

# Merge sales data (nmv only)
pricing_data = pricing_data.merge(
    df_sales[['cohort_id', 'product_id', 'nmv']], 
    on=['cohort_id', 'product_id'], 
    how='left'
)

# Fill nmv with 0 for products without sales
pricing_data['nmv'] = pricing_data['nmv'].fillna(0)

# Merge margin statistics (by cohort_id + product_id)
pricing_data = pricing_data.merge(df_margin_stats, on=['cohort_id', 'product_id'], how='left')

# Merge target margins (by brand + cat)
pricing_data = pricing_data.merge(df_targets, on=['brand', 'cat'], how='left')

# Calculate target_margin (use brand target first, then category target)
pricing_data['target_margin'] = pricing_data['target_bm'].fillna(pricing_data['cat_target_margin']).fillna(0)
pricing_data = pricing_data.drop(columns=['target_bm', 'cat_target_margin'], errors='ignore')

# Fill NaN values with defaults
pricing_data['std'] = pricing_data['std'].fillna(0.01)
pricing_data['avg_margin'] = pricing_data['avg_margin'].fillna(0)

# Merge product groups
pricing_data = pricing_data.merge(df_groups, on='product_id', how='left')

# Remove duplicates
pricing_data = pricing_data.drop_duplicates(subset=['cohort_id', 'product_id'])

print(f"\n{'='*60}")
print(f"PRICING DATA BASE COMPLETE")
print(f"{'='*60}")
print(f"Total records: {len(pricing_data)}")
print(f"\nRecords with market data:")
print(f"  - With marketplace prices: {len(pricing_data[~pricing_data['final_min_price'].isna()])}")
print(f"  - With scrapped prices: {len(pricing_data[~pricing_data['min_scrapped'].isna()])}")
print(f"  - With Ben Soliman prices: {len(pricing_data[~pricing_data['ben_soliman_price'].isna()])}")
print(f"\nRecords with sales (nmv > 0): {len(pricing_data[pricing_data['nmv'] > 0])}")
print(f"Records without sales (nmv = 0): {len(pricing_data[pricing_data['nmv'] == 0])}")


## Group Processing - Fill Missing Prices with Group Medians

In [ ]:
# =============================================================================
# Group Processing - Calculate group-level aggregated prices
# =============================================================================

# Calculate group-level aggregated prices for products with group assignments
groups_data = pricing_data[~pricing_data['group'].isna()].copy()
groups_data['group_nmv'] = groups_data.groupby(['group', 'cohort_id'])['nmv'].transform('sum')
groups_data['cntrb'] = (groups_data['nmv'] / groups_data['group_nmv']).fillna(1)

# Flag if any price/scrapped column is non-NaN
price_cols = [
    'ben_soliman_price', 'final_min_price', 'final_max_price', 'final_mod_price', 'final_true_min', 'final_true_max',
    'min_scrapped', 'scrapped25', 'scrapped50', 'scrapped75', 'max_scrapped'
]
groups_data['flag_non_nan'] = groups_data[price_cols].notna().any(axis=1).astype(int)

# Weighted Median Function
def weighted_median(series, weights):
    valid = ~series.isna() & ~weights.isna()
    s = series[valid]
    w = weights[valid]
    if len(s) == 0:
        return np.nan
    order = np.argsort(s)
    s, w = s.iloc[order], w.iloc[order]
    return s.iloc[np.searchsorted(np.cumsum(w), w.sum() / 2)]

# Perform Weighted Aggregation
groups_agg = (
    groups_data[groups_data['flag_non_nan'] == 1]
    .groupby(['group', 'cohort_id'])
    .apply(lambda g: pd.Series({
        col: weighted_median(g[col], g['cntrb']) for col in price_cols
    }))
    .reset_index()
)

# Fill missing prices with group-level prices
merged = pricing_data.merge(groups_agg, on=['group', 'cohort_id'], how='left', suffixes=('', '_group'))
for col in price_cols:
    merged[col] = merged[col].fillna(merged[f'{col}_group'])

pricing_data = merged.drop(columns=[f'{c}_group' for c in price_cols])

print(f"Pricing data after group processing: {len(pricing_data)} records")

## Price Coverage Filtering

In [ ]:
# =============================================================================
# Price Coverage Filtering - Filter products with sufficient price data
# =============================================================================

# Score price coverage
pricing_data['ben'] = 0
pricing_data['MP'] = 0
pricing_data['sp'] = 0

# Ben Soliman: 1 point if present
pricing_data.loc[~pricing_data['ben_soliman_price'].isna(), 'ben'] = 1

# Marketplace: 1 point if single price, 3 points if range
pricing_data.loc[(pricing_data['final_min_price'] == pricing_data['final_max_price']) & 
                (~pricing_data['final_min_price'].isna()), 'MP'] = 1
pricing_data.loc[(pricing_data['final_min_price'] != pricing_data['final_max_price']) & 
                (~pricing_data['final_min_price'].isna()), 'MP'] = 3

# Scrapped: 1 point if single price, 5 points if range
pricing_data.loc[(pricing_data['min_scrapped'] == pricing_data['max_scrapped']) & 
                (~pricing_data['min_scrapped'].isna()), 'sp'] = 1
pricing_data.loc[(pricing_data['min_scrapped'] != pricing_data['max_scrapped']) & 
                (~pricing_data['min_scrapped'].isna()), 'sp'] = 5

# Total price coverage score
pricing_data['total_p'] = pricing_data['ben'] + pricing_data['MP'] + pricing_data['sp']

# Filter: keep only products with total_p > 2
pricing_data = pricing_data[pricing_data['total_p'] > 2]

print(f"Pricing data after price coverage filtering: {len(pricing_data)} records")

## Price Analysis & Margin Calculation


In [ ]:
# =============================================================================
# Price Analysis Functions
# =============================================================================

def price_analysis(row):
    """Analyze prices and calculate percentiles for a product."""
    wac = row['wac_p']
    avg_margin = row['avg_margin'] if row['avg_margin'] >= 0.01 else row['target_margin']
    std = np.maximum(row['std'], 0.0025)
    target_margin = row['target_margin']
    max_marg = np.maximum(avg_margin, target_margin)
    
    # Collect all price points
    price_list = [
        row['ben_soliman_price'], row['final_min_price'], row['final_mod_price'],
        row['final_max_price'], row['final_true_min'], row['final_true_max'],
        row['min_scrapped'], row['scrapped25'], row['scrapped50'], row['scrapped75'], row['max_scrapped']
    ]
    
    # Filter valid prices within acceptable range
    valid_prices = sorted({
        x for x in price_list 
        if x and not pd.isna(x) and x != 0 
        and wac / (1 - (avg_margin - (10 * std))) <= x <= wac / (1 - (max_marg + 10 * std))
        and x >= wac * 0.9
    })
    
    if not valid_prices:
        return np.nan, np.nan, np.nan, np.nan, np.nan
    
    return (
        np.min(valid_prices),
        np.percentile(valid_prices, 25),
        np.percentile(valid_prices, 50),
        np.percentile(valid_prices, 75),
        np.max(valid_prices)
    )


def calculate_step_bounds(row):
    """Calculate below/above market bounds based on price steps."""
    wac = row['wac_p']
    std = row['std']
    prices = [row['minimum'], row['percentile_25'], row['percentile_50'], row['percentile_75'], row['maximum']]
    
    # Calculate valid steps between price points
    valid_steps = []
    for i in range(len(prices) - 1):
        step = prices[i + 1] - prices[i]
        if (step / wac) <= std * 1.2:
            valid_steps.append(step)
    
    avg_step = np.mean(valid_steps) if valid_steps else min(2 * std, 0.2 * row['target_margin'])
    
    new_min = prices[0] - avg_step if (prices[0] - avg_step) >= wac else prices[0]
    new_max = prices[-1] + avg_step if (prices[-1] + avg_step) >= wac else prices[-1]
    
    return new_min, new_max


In [ ]:
# =============================================================================
# Apply Price Analysis & Margin Calculation
# =============================================================================

# Apply price analysis to calculate price percentiles
pricing_data[['minimum', 'percentile_25', 'percentile_50', 'percentile_75', 'maximum']] = \
    pricing_data.apply(price_analysis, axis=1, result_type='expand')

# Filter out records without valid price analysis
pricing_data = pricing_data[~pricing_data['minimum'].isna()]

# Calculate below/above market bounds
pricing_data[['below_market', 'above_market']] = pricing_data.apply(calculate_step_bounds, axis=1, result_type='expand')

print(f"Pricing data after price analysis: {len(pricing_data)} records")


In [ ]:
# =============================================================================
# Convert prices to margins
# =============================================================================

pricing_data['below_market'] = (pricing_data['below_market'] - pricing_data['wac_p']) / pricing_data['below_market']
pricing_data['market_min'] = (pricing_data['minimum'] - pricing_data['wac_p']) / pricing_data['minimum']
pricing_data['market_25'] = (pricing_data['percentile_25'] - pricing_data['wac_p']) / pricing_data['percentile_25']
pricing_data['market_50'] = (pricing_data['percentile_50'] - pricing_data['wac_p']) / pricing_data['percentile_50']
pricing_data['market_75'] = (pricing_data['percentile_75'] - pricing_data['wac_p']) / pricing_data['percentile_75']
pricing_data['market_max'] = (pricing_data['maximum'] - pricing_data['wac_p']) / pricing_data['maximum']
pricing_data['above_market'] = (pricing_data['above_market'] - pricing_data['wac_p']) / pricing_data['above_market']

# Calculate current margin from current_price
pricing_data['current_margin'] = (pricing_data['current_price'] - pricing_data['wac_p']) / pricing_data['current_price']

# Reorder columns to put product base info first
final_columns = [
    # Product Base Info
    'cohort_id', 'product_id', 'region', 'warehouse_id', 'warehouse', 'sku', 'brand', 'cat',
    # Cost & Price
    'wac1', 'wac_p', 'current_price', 'current_margin',
    # Sales
    'nmv',
    # Market Prices (raw)
    'ben_soliman_price', 
    'final_min_price', 'final_max_price', 'final_mod_price', 'final_true_min', 'final_true_max',
    'min_scrapped', 'scrapped25', 'scrapped50', 'scrapped75', 'max_scrapped',
    # Price Percentiles
    'minimum', 'percentile_25', 'percentile_50', 'percentile_75', 'maximum',
    # Margin Tiers
    'below_market', 'market_min', 'market_25', 'market_50', 'market_75', 'market_max', 'above_market',
    # Supporting Data
    'std', 'avg_margin', 'target_margin', 'group'
]
# Keep only columns that exist in the dataframe
pricing_data = pricing_data[[c for c in final_columns if c in pricing_data.columns]]

print(f"\n{'='*60}")
print(f"PRICING DATA PROCESSING COMPLETE")
print(f"{'='*60}")
print(f"Total records: {len(pricing_data)}")
print(f"\nProduct Base columns:")
print("  - cohort_id, product_id, region, warehouse_id, warehouse, sku, brand, cat, wac1, wac_p, current_price, current_margin")
print(f"\nMargin tier columns:")
print("  - below_market, market_min, market_25, market_50, market_75, market_max, above_market")
print(f"\nSample data:")
pricing_data.head()


## Discount Analysis - Price & Margin After Discount


In [ ]:
# =============================================================================
# Discount Query - Get discount percentage by warehouse and product
# =============================================================================
DISCOUNT_QUERY = f'''
SELECT warehouse_id, product_id, total_discount/total_nmv AS discount_perc
FROM (
    SELECT  
        pso.warehouse_id,
        pso.product_id,
        SUM(pso.total_price) AS total_nmv,
        SUM((ITEM_QUANTITY_DISCOUNT_VALUE * pso.purchased_item_count) + 
            (ITEM_DISCOUNT_VALUE * pso.purchased_item_count)) AS total_discount
    FROM product_sales_order pso 
    JOIN sales_orders so ON so.id = pso.sales_order_id
    WHERE so.created_at::DATE >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 1 
        AND so.sales_order_status_id NOT IN (7, 12)
        AND so.channel IN ('telesales', 'retailer')
        AND pso.purchased_item_count <> 0
    GROUP BY ALL
)
WHERE total_nmv > 0
'''

# Execute discount query
print("Loading discount data...")
df_discount = query_snowflake(DISCOUNT_QUERY)
df_discount = convert_to_numeric(df_discount)
print(f"Loaded {len(df_discount)} discount records")


In [ ]:
# =============================================================================
# Create pricing_with_discount DataFrame
# =============================================================================
print("Creating pricing_with_discount DataFrame...")

# Copy pricing_data
pricing_with_discount = pricing_data.copy()

# Merge discount data (by warehouse_id + product_id)
pricing_with_discount = pricing_with_discount.merge(
    df_discount[['warehouse_id', 'product_id', 'discount_perc']], 
    on=['warehouse_id', 'product_id'], 
    how='left'
)

# Fill missing discount_perc with 0 (no discount)
pricing_with_discount['discount_perc'] = pricing_with_discount['discount_perc'].fillna(0)

# =============================================================================
# Calculate price and margin after discount
# =============================================================================
# Price after discount = current_price * (1 - discount_perc)
pricing_with_discount['price_after_discount'] = (
    pricing_with_discount['current_price'] * (1 - pricing_with_discount['discount_perc'])
)

# Margin after discount = (price_after_discount - wac_p) / price_after_discount
pricing_with_discount['margin_after_discount'] = (
    (pricing_with_discount['price_after_discount'] - pricing_with_discount['wac_p']) / 
    pricing_with_discount['price_after_discount']
)

print(f"\n{'='*60}")
print(f"PRICING WITH DISCOUNT DATA COMPLETE")
print(f"{'='*60}")
print(f"Total records: {len(pricing_with_discount)}")
print(f"Records with discount (discount_perc > 0): {len(pricing_with_discount[pricing_with_discount['discount_perc'] > 0])}")
print(f"Records without discount: {len(pricing_with_discount[pricing_with_discount['discount_perc'] == 0])}")
print(f"\nNew columns added:")
print("  - discount_perc: discount percentage from sales")
print("  - price_after_discount: current_price * (1 - discount_perc)")
print("  - margin_after_discount: (price_after_discount - wac_p) / price_after_discount")
print(f"\nSample data with discounts:")
pricing_with_discount[pricing_with_discount['discount_perc'] > 0][
    ['product_id', 'warehouse_id', 'current_price', 'current_margin', 
     'discount_perc', 'price_after_discount', 'margin_after_discount']
].head(10)


In [ ]:
# =============================================================================
# Price Position - Determine where price_after_discount falls in market tiers
# =============================================================================

def get_price_position(row):
    """Determine the price position relative to market price tiers."""
    price = row['price_after_discount']
    wac = row['wac_p']
    
    # Check if we have market data (minimum price exists)
    if pd.isna(row['minimum']) or pd.isna(price):
        return "No Market Data"
    
    # Get price tiers
    min_price = row['minimum']
    p25 = row['percentile_25']
    p50 = row['percentile_50']
    p75 = row['percentile_75']
    max_price = row['maximum']
    
    # Calculate below_market and above_market prices from margins
    # margin = (price - wac) / price  =>  price = wac / (1 - margin)
    below_market_margin = row['below_market']
    above_market_margin = row['above_market']
    
    below_market_price = wac / (1 - below_market_margin) if below_market_margin < 1 else min_price
    above_market_price = wac / (1 - above_market_margin) if above_market_margin < 1 else max_price
    
    # Determine position based on price tiers
    if price < below_market_price:
        return "Below Market"
    elif price < min_price:
        return "Below Min"
    elif price < p25:
        return "At Min"
    elif price < p50:
        return "At 25th"
    elif price < p75:
        return "At 50th"
    elif price < max_price:
        return "At 75th"
    elif price < above_market_price:
        return "At Max"
    else:
        return "Above Market"

# Apply price position function
pricing_with_discount['price_position'] = pricing_with_discount.apply(get_price_position, axis=1)

# Summary of price positions
print(f"\n{'='*60}")
print(f"PRICE POSITION ANALYSIS")
print(f"{'='*60}")
print("\nPrice Position Distribution:")
print(pricing_with_discount['price_position'].value_counts().to_string())
print(f"\nPrice Position Percentages:")
print((pricing_with_discount['price_position'].value_counts(normalize=True) * 100).round(2).astype(str) + '%')

# Sample data showing price position
print(f"\nSample data with price position:")
pricing_with_discount[
    ['product_id', 'warehouse_id', 'sku', 'current_price', 'discount_perc', 
     'price_after_discount', 'minimum', 'maximum', 'price_position']
].head(15)


In [ ]:
# =============================================================================
# Stock Query - Get available stock by warehouse and product
# =============================================================================
STOCK_QUERY = '''
SELECT 
    pw.warehouse_id,
    pw.product_id,
    pw.available_stock::INTEGER AS stocks
FROM product_warehouse pw
WHERE pw.warehouse_id NOT IN (6, 9, 10)
    AND pw.is_basic_unit = 1
'''

# Execute stock query
print("Loading stock data...")
df_stocks = query_snowflake(STOCK_QUERY)
df_stocks = convert_to_numeric(df_stocks)
print(f"Loaded {len(df_stocks)} stock records")

# Merge stock data with pricing_with_discount
pricing_with_discount = pricing_with_discount.merge(
    df_stocks[['warehouse_id', 'product_id', 'stocks']], 
    on=['warehouse_id', 'product_id'], 
    how='left'
)

# Fill missing stocks with 0
pricing_with_discount['stocks'] = pricing_with_discount['stocks'].fillna(0).astype(int)

print(f"\nStock data merged!")
print(f"Records with stock (stocks > 0): {len(pricing_with_discount[pricing_with_discount['stocks'] > 0])}")
print(f"Records without stock (stocks = 0): {len(pricing_with_discount[pricing_with_discount['stocks'] == 0])}")
print(f"\nSample data with stocks:")
pricing_with_discount[
    ['product_id', 'warehouse_id', 'sku', 'stocks', 'price_after_discount', 'price_position']
].head(10)


In [ ]:
# =============================================================================
# Zero Demand Query - Identify SKUs with zero/low demand
# =============================================================================
ZERO_DEMAND_QUERY = f'''
WITH last_oss AS (
    SELECT product_id, warehouse_id, TIMESTAMP AS last_in_stock_day
    FROM (
        SELECT *, ROW_NUMBER() OVER(PARTITION BY product_id, warehouse_id ORDER BY TIMESTAMP DESC) AS rnk 
        FROM materialized_views.STOCK_DAY_CLOSE
        WHERE AVAILABLE_STOCK = 0 
            AND TIMESTAMP >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 120
        QUALIFY rnk = 1 
    )
),

current_stocks AS (
    SELECT product_id, warehouse_id, AVAILABLE_STOCK, activation
    FROM PRODUCT_WAREHOUSE
    WHERE IS_BASIC_UNIT = 1
        AND CASE WHEN product_id = 1309 THEN packing_unit_id <> 23 ELSE TRUE END
),

prs AS (
    SELECT DISTINCT 
        product_purchased_receipts.product_id,
        purchased_receipts.warehouse_id,
        purchased_receipts.date::DATE AS date,
        product_purchased_receipts.purchased_item_count * product_purchased_receipts.basic_unit_count AS purchase_min_count
    FROM product_purchased_receipts
    JOIN purchased_receipts ON purchased_receipts.id = product_purchased_receipts.purchased_receipt_id
    JOIN last_oss lo ON product_purchased_receipts.product_id = lo.product_id 
        AND lo.warehouse_id = purchased_receipts.warehouse_id 
        AND purchased_receipts.date > lo.last_in_stock_day 
    WHERE product_purchased_receipts.purchased_item_count <> 0
        AND purchased_receipts.purchased_receipt_status_id IN (4, 5, 7)
        AND purchased_receipts.date::DATE >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 120
),

main AS (
    SELECT 
        prs.product_id, 
        prs.warehouse_id, 
        MIN(date) AS first_order_date, 
        SUM(purchase_min_count) AS total_recieved, 
        cs.AVAILABLE_STOCK, 
        cs.activation
    FROM prs 
    JOIN current_stocks cs ON cs.product_id = prs.product_id AND prs.warehouse_id = cs.warehouse_id
    GROUP BY prs.product_id, prs.warehouse_id, cs.AVAILABLE_STOCK, cs.activation
),

sold_days AS (
    SELECT product_id, warehouse_id, COUNT(DISTINCT o_date) AS sales_days
    FROM (
        SELECT DISTINCT
            so.created_at::DATE AS o_date,
            pso.warehouse_id,
            pso.product_id,
            SUM(pso.purchased_item_count * basic_unit_count) AS daily_qty
        FROM product_sales_order pso
        JOIN sales_orders so ON so.id = pso.sales_order_id
        JOIN main m ON m.product_id = pso.product_id 
            AND m.warehouse_id = pso.warehouse_id 
            AND so.created_at::DATE >= m.first_order_date
        WHERE so.created_at::DATE BETWEEN CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 120 
            AND CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE
            AND so.sales_order_status_id NOT IN (7, 12)
            AND so.channel IN ('telesales', 'retailer')
            AND pso.purchased_item_count <> 0
        GROUP BY o_date, pso.warehouse_id, pso.product_id
    )
    GROUP BY product_id, warehouse_id
)

SELECT DISTINCT warehouse_id, product_id
FROM (
    SELECT m.product_id, m.warehouse_id, m.first_order_date, m.activation,
        COALESCE(sd.sales_days, 0) AS sales_days,
        COALESCE(sd.sales_days, 0)::FLOAT / NULLIF((CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 1) - m.first_order_date, 0) AS perc_days
    FROM main m 
    LEFT JOIN sold_days sd ON sd.product_id = m.product_id AND sd.warehouse_id = m.warehouse_id
    WHERE m.first_order_date < CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 10
)
WHERE perc_days <= 0.3
    AND activation = 'true'
'''

# Execute zero demand query
print("Loading zero demand SKUs...")
df_zero_demand = query_snowflake(ZERO_DEMAND_QUERY)
df_zero_demand = convert_to_numeric(df_zero_demand)
print(f"Loaded {len(df_zero_demand)} zero demand SKU records")


In [ ]:
# =============================================================================
# Add Zero Demand Flag to pricing_with_discount
# =============================================================================

# Add a marker column to identify zero demand SKUs
df_zero_demand['zero_demand'] = 1

# Merge with pricing_with_discount
pricing_with_discount = pricing_with_discount.merge(
    df_zero_demand[['warehouse_id', 'product_id', 'zero_demand']], 
    on=['warehouse_id', 'product_id'], 
    how='left'
)

# Fill missing values with 0 (not zero demand)
pricing_with_discount['zero_demand'] = pricing_with_discount['zero_demand'].fillna(0).astype(int)

print(f"Zero demand flag added!")
print(f"SKUs flagged as zero demand: {len(pricing_with_discount[pricing_with_discount['zero_demand'] == 1])}")
print(f"SKUs with normal demand: {len(pricing_with_discount[pricing_with_discount['zero_demand'] == 0])}")


In [ ]:
# =============================================================================
# OOS Yesterday Query - Identify SKUs out of stock yesterday
# =============================================================================
OOS_YESTERDAY_QUERY = f'''
SELECT DISTINCT product_id, warehouse_id,
    CASE WHEN opening_stocks = 0 AND closing_stocks = 0 THEN 1
         ELSE 0 
    END AS oos_yesterday
FROM (
    SELECT 
        timestamp,
        product_id,
        warehouse_id, 
        AVAILABLE_STOCK AS closing_stocks,
        LAG(AVAILABLE_STOCK) OVER (PARTITION BY product_id, warehouse_id ORDER BY TIMESTAMP) AS opening_stocks
    FROM materialized_views.stock_day_close
    WHERE timestamp::DATE >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 2
    QUALIFY opening_stocks IS NOT NULL
)
WHERE oos_yesterday = 1
'''

# Execute OOS yesterday query
print("Loading OOS yesterday data...")
df_oos_yesterday = query_snowflake(OOS_YESTERDAY_QUERY)
df_oos_yesterday = convert_to_numeric(df_oos_yesterday)
print(f"Loaded {len(df_oos_yesterday)} OOS yesterday records")


In [ ]:
# =============================================================================
# Add OOS Yesterday Flag to pricing_with_discount
# =============================================================================

# Merge with pricing_with_discount
pricing_with_discount = pricing_with_discount.merge(
    df_oos_yesterday[['warehouse_id', 'product_id', 'oos_yesterday']], 
    on=['warehouse_id', 'product_id'], 
    how='left'
)

# Fill missing values with 0 (not OOS yesterday)
pricing_with_discount['oos_yesterday'] = pricing_with_discount['oos_yesterday'].fillna(0).astype(int)

print(f"OOS yesterday flag added!")
print(f"SKUs out of stock yesterday: {len(pricing_with_discount[pricing_with_discount['oos_yesterday'] == 1])}")
print(f"SKUs in stock yesterday: {len(pricing_with_discount[pricing_with_discount['oos_yesterday'] == 0])}")


In [ ]:
# =============================================================================
# Running Rate Query - Get in-stock running rate by warehouse and product
# =============================================================================
RUNNING_RATE_QUERY = f'''
WITH params AS (
    SELECT
        CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE AS run_date,
        DATEADD(month, -3, CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE) AS history_start
),

-- Daily sales aggregation
sales_base AS (
    SELECT
        pso.product_id,
        pso.warehouse_id,
        DATE_TRUNC('day', pso.created_at)::DATE AS date,
        SUM(pso.purchased_item_count * pso.basic_unit_count) AS sold_units,
        SUM(pso.purchased_item_count * pso.basic_unit_count * pso.item_price)
            / NULLIF(SUM(pso.purchased_item_count * pso.basic_unit_count), 0) AS avg_selling_price,
        COUNT(DISTINCT so.retailer_id) AS retailer_count
    FROM product_sales_order pso
    JOIN sales_orders so ON pso.sales_order_id = so.id
    WHERE DATE_TRUNC('day', pso.created_at)::DATE >= (SELECT history_start FROM params)
    GROUP BY 1, 2, 3
),

-- Stock daily metrics
stock_daily AS (
    SELECT
        product_id,
        warehouse_id,
        DATE_TRUNC('day', TIMESTAMP)::DATE AS date,
        MAX_BY(available_stock, TIMESTAMP) AS stock_closing,
        24 * SUM(CASE WHEN activation = FALSE OR available_stock = 0 THEN 1 ELSE 0 END)::FLOAT 
            / NULLIF(COUNT(*), 0) AS oos_hours,
        MAX(CASE WHEN activation = TRUE AND available_stock > 0 THEN 1 ELSE 0 END) AS in_stock_flag
    FROM materialized_views.STOCK_SNAP_SHOTS_RECENT
    WHERE product_id IS NOT NULL
    GROUP BY product_id, warehouse_id, date
),

-- Join sales + stock + WAC (only in-stock days)
base_data AS (
    SELECT
        sb.product_id,
        sb.warehouse_id,
        sb.date,
        sb.sold_units,
        sb.avg_selling_price,
        sb.retailer_count,
        sd.oos_hours,
        sd.in_stock_flag,
        ac.wac_p AS wac,
        CASE WHEN DAYOFWEEKISO(sb.date) IN (5, 6) THEN 1 ELSE 0 END AS is_weekend
    FROM sales_base sb
    LEFT JOIN stock_daily sd ON sb.product_id = sd.product_id 
        AND sb.warehouse_id = sd.warehouse_id AND sb.date = sd.date
    LEFT JOIN finance.ALL_COGS ac ON sb.product_id = ac.product_id 
        AND sb.date BETWEEN ac.from_date AND ac.to_date
    WHERE sd.in_stock_flag = 1
),

-- Stats per SKU x Warehouse
sku_wh_stats AS (
    SELECT
        product_id, warehouse_id,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY sold_units) AS med_units,
        PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY sold_units) AS pct95_units,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY retailer_count) AS med_retailers,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY 
            CASE WHEN avg_selling_price IS NULL OR avg_selling_price = 0 THEN 0 
            ELSE (avg_selling_price - COALESCE(wac, 0)) / NULLIF(avg_selling_price, 0) END
        ) AS med_margin
    FROM base_data
    GROUP BY product_id, warehouse_id
),

-- Cap outliers and adjust for retailer spikes
adjusted AS (
    SELECT
        b.product_id, b.warehouse_id, b.date, b.in_stock_flag, b.oos_hours, b.is_weekend,
        b.avg_selling_price, b.wac, s.med_margin,
        CASE 
            WHEN b.retailer_count > GREATEST(2, s.med_retailers * 2) 
                AND b.retailer_count > 0 AND s.med_retailers IS NOT NULL
            THEN ROUND(LEAST(b.sold_units, s.pct95_units) * (s.med_retailers::FLOAT / NULLIF(b.retailer_count::FLOAT, 0)), 0)
            ELSE LEAST(b.sold_units, s.pct95_units)
        END AS units_adjusted
    FROM base_data b
    LEFT JOIN sku_wh_stats s ON b.product_id = s.product_id AND b.warehouse_id = s.warehouse_id
),

-- Apply weights (recency, stock availability, weekend, margin)
weighted AS (
    SELECT
        product_id, warehouse_id, date, units_adjusted,
        (
            -- Recency weight
            CASE WHEN date >= DATEADD(day, -21, (SELECT run_date FROM params)) THEN 1.5
                 WHEN date >= DATEADD(day, -90, (SELECT run_date FROM params)) THEN 1.0
                 ELSE 0.5 END
            -- In-stock weight
            * CASE WHEN in_stock_flag = 1 AND COALESCE(oos_hours, 0) < 12 THEN 1.4
                   WHEN in_stock_flag = 1 AND COALESCE(oos_hours, 0) >= 12 THEN 0.9
                   ELSE 0.6 END
            -- Weekend weight
            * CASE WHEN is_weekend = 1 THEN 0.7 ELSE 1.0 END
            -- Margin weight
            * CASE WHEN avg_selling_price IS NULL OR avg_selling_price = 0 OR med_margin IS NULL THEN 1.0
                   WHEN ((avg_selling_price - COALESCE(wac, 0)) / NULLIF(avg_selling_price, 0)) < med_margin
                   THEN 1.0 + LEAST((med_margin - ((avg_selling_price - COALESCE(wac, 0)) / NULLIF(avg_selling_price, 0))) * 2.0, 0.6)
                   WHEN ((avg_selling_price - COALESCE(wac, 0)) / NULLIF(avg_selling_price, 0)) > med_margin
                   THEN 1.0 - LEAST((((avg_selling_price - COALESCE(wac, 0)) / NULLIF(avg_selling_price, 0)) - med_margin) * 2.0, 0.4)
                   ELSE 1.0 END
        ) AS final_weight
    FROM adjusted
    WHERE units_adjusted IS NOT NULL
),

-- Weighted average forecast
forecast_base AS (
    SELECT
        product_id, warehouse_id,
        SUM(units_adjusted * final_weight) / NULLIF(SUM(final_weight), 0) AS weighted_avg_units
    FROM weighted
    GROUP BY product_id, warehouse_id
),

-- Zero-sales last 4 days (with stock) exclusion flag
last4_flag AS (
    SELECT product_id, warehouse_id,
        CASE WHEN COUNT(*) = 4 
             AND SUM(CASE WHEN COALESCE(sold_units, 0) = 0 AND in_stock_flag = 1 THEN 1 ELSE 0 END) = 4
        THEN 1 ELSE 0 END AS exclude_flag
    FROM base_data
    WHERE date >= DATEADD(day, -4, (SELECT run_date FROM params)) 
        AND date < (SELECT run_date FROM params)
    GROUP BY product_id, warehouse_id
),

-- Zero sales excluded (in stock but no sales)
zero_sales_excluded AS (
    SELECT DISTINCT s.warehouse_id, s.product_id
    FROM (
        SELECT pw.warehouse_id, pw.product_id, SUM(pw.available_stock)::INT AS stocks
        FROM product_warehouse pw
        WHERE pw.warehouse_id NOT IN (6, 9, 10) AND pw.is_basic_unit = 1 AND pw.available_stock > 0
        GROUP BY pw.warehouse_id, pw.product_id
    ) s
    LEFT JOIN (
        SELECT pso.product_id, pso.warehouse_id, SUM(pso.total_price) AS nmv
        FROM product_sales_order pso
        JOIN sales_orders so ON so.id = pso.sales_order_id
        WHERE so.created_at::date BETWEEN CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 5 
            AND CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 1
            AND so.sales_order_status_id NOT IN (7, 12) AND so.channel IN ('telesales', 'retailer')
            AND pso.purchased_item_count <> 0
        GROUP BY pso.product_id, pso.warehouse_id
    ) md ON md.product_id = s.product_id AND md.warehouse_id = s.warehouse_id
    LEFT JOIN finance.all_cogs f ON f.product_id = s.product_id
        AND f.from_date::date <= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE
        AND f.to_date::date > CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE
    LEFT JOIN (
        SELECT pr.warehouse_id, ppr.product_id, SUM(ppr.final_price) AS total_prs
        FROM product_purchased_receipts ppr
        JOIN purchased_receipts pr ON pr.id = ppr.purchased_receipt_id
        WHERE pr.date::date >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 4
            AND pr.is_actual = 'true' AND pr.purchased_receipt_status_id IN (4, 5, 7)
            AND ppr.purchased_item_count <> 0
        GROUP BY pr.warehouse_id, ppr.product_id
    ) prs ON prs.product_id = s.product_id AND prs.warehouse_id = s.warehouse_id
    WHERE COALESCE(md.nmv, 0) = 0 
        AND COALESCE(prs.total_prs, 0) < 0.7 * (COALESCE(f.wac_p, 0) * s.stocks)
),

-- First sale date for new products
first_sale AS (
    SELECT product_id, warehouse_id, MIN(date) AS first_sale_date
    FROM base_data WHERE sold_units > 0
    GROUP BY product_id, warehouse_id
)

-- Final output: running rate per warehouse/product
SELECT
    fb.warehouse_id,
    fb.product_id,
    CASE
        WHEN l4.exclude_flag = 1 THEN 0
        WHEN fs.first_sale_date >= DATEADD(day, -2, (SELECT run_date FROM params))
        THEN GREATEST(CEIL(fb.weighted_avg_units), 1)
        ELSE CEIL(fb.weighted_avg_units)
    END AS In_stock_rr
FROM forecast_base fb
LEFT JOIN last4_flag l4 ON fb.product_id = l4.product_id AND fb.warehouse_id = l4.warehouse_id
LEFT JOIN first_sale fs ON fb.product_id = fs.product_id AND fb.warehouse_id = fs.warehouse_id
LEFT JOIN zero_sales_excluded zse ON fb.product_id = zse.product_id AND fb.warehouse_id = zse.warehouse_id
WHERE zse.product_id IS NULL
'''

# Execute running rate query
print("Loading running rate data (this may take a moment)...")
df_running_rate = query_snowflake(RUNNING_RATE_QUERY)
df_running_rate = convert_to_numeric(df_running_rate)
print(f"Loaded {len(df_running_rate)} running rate records")


In [ ]:
# =============================================================================
# Merge Running Rate and Calculate DOH (Days on Hand)
# =============================================================================

# Merge running rate data with pricing_with_discount
pricing_with_discount = pricing_with_discount.merge(
    df_running_rate[['warehouse_id', 'product_id', 'in_stock_rr']], 
    on=['warehouse_id', 'product_id'], 
    how='left'
)

# Fill missing running rate with 0
pricing_with_discount['in_stock_rr'] = pricing_with_discount['in_stock_rr'].fillna(0)

# Calculate DOH (Days on Hand) = stocks / in_stock_rr
# Handle division by zero - if running rate is 0, DOH is infinite (use 999)
pricing_with_discount['doh'] = np.where(
    pricing_with_discount['in_stock_rr'] > 0,
    pricing_with_discount['stocks'] / pricing_with_discount['in_stock_rr'],
    999
)
